# Armed Conflict Location Event Data

"The Armed Conflict Location & Event Data Project (ACLED) is a disaggregated data collection, analysis, and crisis mapping project. ACLED collects the dates, actors, locations, fatalities, and types of all reported political violence and protest events across Africa, the Middle East, Latin America & the Caribbean, East Asia, South Asia, Southeast Asia, Central Asia & the Caucasus, Europe, and the United States of America. The ACLED team conducts analysis to describe, explore, and test conflict scenarios, and makes both data and analysis open for free use by the public."

"ACLED is a registered non-profit organization with 501(c)(3) status in the United States."

Copied from [about ACLED](https://acleddata.com/about-acled/)

[ACLED website](https://acleddata.com/#/dashboard)

# 0. Import Python Dependencies and Enter Access Information


In [19]:
#import libraries
import requests
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact
import json
from bokeh.tile_providers import get_provider, Vendors
from pyproj import Transformer
tile_provider = get_provider('STAMEN_TERRAIN')

############################################################
#Code to Input Data
##########################################################

print("Please enter your ACLED Email and API Key")
key_input = widgets.Text(
    placeholder = "Input API Key",
    description="API Key", 
    disabled = False)

def api_in(key):
    key_entered = key
    return key_entered

key_entered = interact(api_in, key = key_input)

email_input = widgets.Text(
    placeholder = "Input ACLED email",
    description="Email", 
    disabled = False)

def email_in(email):
    email_entered = email
    return email_entered

email_entered = interact(email_in, email = email_input)
#NIU_Library@dodiis.mil
#4Uoeeh94QslgoIucWFc3

ModuleNotFoundError: No module named 'pyproj'

# 1. Determine How You Would like to Make the Request

If you are able to code your own requests skip to step 3 and use the API Guide located in this repo. 

In [2]:
#Read in the coverage
docs = pd.read_csv("ACLED-Country-Coverage-and-ISO-Codes_3.2020.csv")
docs =docs.dropna()
# Conver to lists for widgets
country = sorted(list(docs['Country']))
date = list(docs["Start Date"])
iso = list(docs["ISO Code"])
region_dict = {"Western Africa":1, "Middle Africa": 2,"Eastern Africa": 3,"Southern Africa": 4,"Northern Africa": 5, "South Asia": 7, "Western Asia":8,"Southeast Asia":9,
          "Middle East": 11,"Europe":12,"Caucasus and Central Asia":13,"Central America":14,"South America":15,"Caribbean":16,
          "East Asia":17}

print("Do you want to look by country or region?")

approach = widgets.Select(options = ["By Country", "By Region", "By Actor", "By Event Type"],
                          description="Selection Approach", disabled=False)

def update_app(approach):
    approach_selection = approach
    return approach


approach_selection = interact(update_app, approach = approach)



Do you want to look by country or region?


interactive(children=(Select(description='Selection Approach', options=('By Country', 'By Region', 'By Actor',…

# 2. From your request either select or type in your requests and enter your email and key

In [3]:
if approach.value == "By Country":
    print("Select multiple countries in a row with shift or numerous individual countries with ctrl/cmd")
    country_select = widgets.SelectMultiple(options = country,
    description='Country',
    disabled=False)

    country_selected = []
    def update(country):
        country = list(country)
        return country

    country_selection = interact(update, country= country_select)

elif approach.value == "By Region": 
    print("Select multiple region in a row with shift or numerous individual regions comd/ctrl") 
    region_select = widgets.SelectMultiple(
        options = list(region_dict.keys()),description="Regions", 
        disabled = False)


    region_choice = []
    def reg_update(reg):
        region_selected = reg
        return region_selected

    region_selection = interact(reg_update, reg = region_select)
    
elif approach.value == "By Actor":
    print("Seperate different actor types with a comma")
    actor_select = widgets.Text(
        placeholder = "Type Actor Here",
        description="Actor", 
        disabled = False)


    actor_choice = []
    def actor_update(actor):
        actor_selected = actor
        return actor_selected
    
    actor_selection = interact(actor_update, actor = actor_select)
    


Select multiple countries in a row with shift or numerous individual countries with ctrl/cmd


interactive(children=(SelectMultiple(description='Country', options=('Afghanistan', 'Albania', 'Algeria', 'Ang…

# 3. Create the request and See Size 

In [4]:
api_key = key_input.value
email = email_input.value
api_list = []
ref_list = []

if approach.value == "By Country":
    for cntry in country_select.value: 
        #Ping ACLED to determine size of request
        size = 'https://api.acleddata.com/country/read?key={}&email={}&country={}&country_where=%3D'.format(api_key,email,cntry)
        resp = requests.get(size)
        resp = json.loads(resp.text)
        print("Your request for {} contains {} records from {} to {}.\n".format(cntry, resp['data'][0]['event_count'],
                                                                             resp['data'][0]['first_event_date'], 
                                                                             resp['data'][0]['last_event_date']))
        #Create request URL
        api_url = 'https://api.acleddata.com/acled/read?key={}&email={}&country={}&country_where=%3D'.format(api_key,email,cntry)
        api_list.append(api_url)
        ref_list.append(cntry)
    print("Your request urls are {}".format(api_list))
        
elif approach.value == "By Region": 
    for reg in region_select.value:
        #Ping ACLED to determine size of request
        size = 'https://api.acleddata.com/region/read?key={}&email={}&region={}'.format(api_key,email,region_dict[reg])
        resp = requests.get(size)
        resp = json.loads(resp.text)
        print("Your request for {} contains {} records from {} to {}.\n".format(reg, resp['data'][0]['event_count'],
                                                                             resp['data'][0]['first_event_date'], 
                                                                             resp['data'][0]['last_event_date']))
        #Create request URL
        api_url = 'https://api.acleddata.com/acled/read?key={}&email={}&region={}'.format(api_key,email,region_dict[reg])
        api_list.append(api_url)
        ref_list.append(reg)
    print("Your request urls are {}".format(api_list))
    
elif approach.value == "By Actor":
    actor_split =actor_select.split(',')
    for act in actor_split:
        #Ping ACLED to detemrine size of request
        size = 'https://api.acleddata.com/actpr/read?key={}&email={}&actor={}.\n'.format(api_key,email,act)
        resp = requests.get(size)
        resp = json.laods(resp.text)
        print("Your request for {} contains {} records from {} to {}.".format(reg, resp['data'][0]['event_count'],
                                                                             resp['data'][0]['first_event_date'], 
                                                                             resp['data'][0]['last_event_date']))
        #Create Request URL
        api_url = 'https://api.acleddata.com/acled/read?key={}&email={}?actor={}'.format(api_key,email,act)
        api_list.append(api_url)
        ref_list.append(act)
    print("Your request urls are {}".format(api_list)) #creates request according to ACLED format specifications - p. 13
    
print("\nIf your requests total more than 15,000 records we recommend reducing your request.")

Your request for Albania contains 827 records from 2018-01-12 to 2021-01-26.

Your request urls are ['https://api.acleddata.com/acled/read?key=4Uoeeh94QslgoIucWFc3&email=NIU_Library@dodiis.mil&country=Albania&country_where=%3D']

If your requests total more than 15,000 records we recommend reducing your request.


# 4. Send the Request

In [5]:

results = {} # empty data strcture to store results
num_results = 500 # condition to continue adding pages
count = 0  # tracker of results
page = 1 #Per the documentation each page will give us more results

for idx in range(len(ref_list)): 
    results[ref_list[idx]] = []
    page=1
    while num_results == 500: #if less 500 or 0 we know we have all the results
        print ("starting ", page, " ", num_results) #just to see our progress
        data = requests.get(api_list[idx]+"&page={}".format(page)) #call the previous function 
        data = data.json()
        results[ref_list[idx]].append(data['data']) #store in our results
        count += data['count'] #Track number of results
        num_results = data['count'] #update our condition
        page += 1 #update our page variable
        print ("Total Results ", count) #Track our progress
        break
    



starting  1   500
Total Results  500


# 5. Consolidate the Results

The next cell takes the results and place them in a giant table/DataFrame. Based on the user search. Users can select a result from the drop down they want to see. 

In [6]:
#Now I want to put them together into one giant result
res_consolidated = {}
for req,data in results.items(): 
    res_consolidated[req] = [] 
    for d in data: 
        res_consolidated[req] += d
    df = pd.DataFrame(res_consolidated[req])
    res_consolidated[req] = df

res_select = widgets.Dropdown(options = list(res_consolidated.keys()),
    description='Results',
    disabled=False)

def update(results):
    return res_consolidated[results]

res_selection = interact(update, results= res_select)


interactive(children=(Dropdown(description='Results', options=('Albania',), value='Albania'), Output()), _dom_…

In [6]:
#Now I want to put them together into one giant result
res_consolidated = {}
for req,data in results.items(): 
    res_consolidated[req] = [] 
    for d in data: 
        res_consolidated[req] += d
    df = pd.DataFrame(res_consolidated[req])
    res_consolidated[req] = df

res_select = widgets.Dropdown(options = list(res_consolidated.keys()),
    description='Results',
    disabled=False)

def update(results):
    return res_consolidated[results]

res_selection = interact(update, results= res_select)


interactive(children=(Dropdown(description='Results', options=('Albania',), value='Albania'), Output()), _dom_…

# 6. Select a Sub Category



In [14]:
#Now I want to put them together into one giant result
type_select = widgets.Dropdown(options = list(res_consolidated[res_select.value].columns),
    description='Sub Category',
    disabled=False)

def update(sub):
    return sub

res_selection = interact(update, sub= type_select)


interactive(children=(Dropdown(description='Sub Category', options=('data_id', 'iso', 'event_id_cnty', 'event_…

# 7. Select if you to select within the subcategory. 



In [15]:
#Now I want to put them together into one giant result
sub_select = widgets.Dropdown(options = list(res_consolidated[res_select.value][type_select.value].unique()),
    description='Category Select',
    disabled=False)

def update(cat):
    return cat

res_selection = interact(update, cat= sub_select)


interactive(children=(Dropdown(description='Category Select', options=('Europe',), value='Europe'), Output()),…

0             Makati
1        Quezon City
2          San Pedro
3         Bagumbayan
4         Davao City
            ...     
11507         Bacuag
11508         Manila
11509           Pata
11510        Pagatin
11511            Goa
Name: location, Length: 11512, dtype: object